# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,10.91,82,0,1.03,SJ,1722476273
1,1,punta arenas,-53.1500,-70.9167,4.06,93,75,7.72,CL,1722476275
2,2,bilibino,68.0546,166.4372,8.37,40,23,1.09,RU,1722476277
3,3,minas de marcona,-15.2119,-75.1103,12.85,79,9,2.22,PE,1722476279
4,4,strezhevoy,60.7333,77.5889,17.70,75,14,1.67,RU,1722476281


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500, 
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df = city_data_df.loc[
    ((city_data_df["Max Temp"] <= 30.0) & (city_data_df["Max Temp"] >= 20.0))\
        & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna(how="any")

# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,adamstown,-25.0660,-130.1015,22.24,87,0,9.13,PN,1722476306
19,19,rurrenabaque,-14.4413,-67.5278,22.26,64,0,0.86,BO,1722476310
49,49,remire-montjoly,4.9167,-52.2667,25.02,94,0,1.54,GF,1722476370
60,60,hawaiian paradise park,19.5933,-154.9731,27.73,70,0,4.63,US,1722476393
73,73,waddan,29.1614,16.1390,25.63,52,0,0.04,LY,1722476418


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
%%capture --no-display
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
17,adamstown,PN,-25.0660,-130.1015,87,
19,rurrenabaque,BO,-14.4413,-67.5278,64,
49,remire-montjoly,GF,4.9167,-52.2667,94,
60,hawaiian paradise park,US,19.5933,-154.9731,70,
73,waddan,LY,29.1614,16.1390,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
rurrenabaque - nearest hotel: Hostal Madidi
remire-montjoly - nearest hotel: Complexe Belova
hawaiian paradise park - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
hernando - nearest hotel: Holiday Inn Express & Suites Brooksville West
hilo - nearest hotel: Dolphin Bay Hotel
argostolion - nearest hotel: Ainos Hotel
ar rutbah - nearest hotel: No hotel found
whitefish - nearest hotel: The Firebrand Hotel
xunchang - nearest hotel: No hotel found
fredericksburg - nearest hotel: Kenmore Inn
tyup - nearest hotel: No hotel found
zeya - nearest hotel: Серебряный створ
greenville - nearest hotel: Quality Inn
kenai - nearest hotel: Beluga Lookout Hotel
griffith - nearest hotel: Quality Inn & Suites Hammond
brignoles - nearest hotel: Le Provence
broome - nearest hotel: No hotel found
santana - nearest hotel: Hotel Teodoro
ursulo galvan - nearest hotel: Las Fuentes del B

,City,Country,Lat,Lng,Humidity,Hotel Name
17,adamstown,PN,-25.0660,-130.1015,87,No hotel found
19,rurrenabaque,BO,-14.4413,-67.5278,64,Hostal Madidi
49,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
60,hawaiian paradise park,US,19.5933,-154.9731,70,No hotel found
73,waddan,LY,29.1614,16.1390,52,No hotel found
82,carnarvon,AU,-24.8667,113.6333,71,No hotel found
94,hernando,US,28.5419,-82.4582,84,Holiday Inn Express & Suites Brooksville West
129,hilo,US,19.7297,-155.0900,72,Dolphin Bay Hotel
131,argostolion,GR,38.1811,20.4890,31,Ainos Hotel
151,ar rutbah,IQ,33.0381,40.2848,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriImagery",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)